<a href="https://colab.research.google.com/github/joash-alonso/joash-alonso/blob/main/EncoderOnlyTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.5/573.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 924.6/924.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 43.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic 2.2.1 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.6.1 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras_nlp
import tensorflow as tf
import keras_core as keras
from typing import Tuple, Dict

physical_devices = tf.config.list_physical_devices("GPU")
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

**SST-2**: a text classification dataset and our "end goal". This dataset is often used to benchmark language models.

**WikiText-103**: A medium sized collection of featured articles from English Wikipedia, which we will use for pretraining.

In [ ]:
# Download pretraining data.
keras.utils.get_file(
    origin="https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip",
    extract=True,
)
wiki_dir = os.path.expanduser("~/.keras/datasets/wikitext-103-raw/")

# Download finetuning data.
keras.utils.get_file(
    origin="https://dl.fbaipublicfiles.com/glue/data/SST-2.zip",
    extract=True,
)
sst_dir = os.path.expanduser("~/.keras/datasets/SST-2/")

# Download vocabulary data.
vocab_file = keras.utils.get_file(
    origin="https://storage.googleapis.com/tensorflow/keras-nlp/examples/bert/bert_vocab_uncased.txt",
)

191984949/191984949 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
7439277/7439277 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
231508/231508 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step       


In [ ]:
# Preprocessing params.
PRETRAINING_BATCH_SIZE = 128
FINETUNING_BATCH_SIZE = 32
SEQ_LENGTH = 128
MASK_RATE = 0.25
PREDICTIONS_PER_SEQ = 32

In [ ]:
# Load SST-2.
sst_train_ds = tf.data.experimental.CsvDataset(
    sst_dir + "train.tsv", [tf.string, tf.int32], header=True, field_delim="\t"
).batch(FINETUNING_BATCH_SIZE)

sst_val_ds = tf.data.experimental.CsvDataset(
    sst_dir + "dev.tsv", [tf.string, tf.int32], header=True, field_delim="\t"
).batch(FINETUNING_BATCH_SIZE)

# Load wikitext-103 and filter out short lines.
wiki_train_ds = (
    tf.data.TextLineDataset(wiki_dir + "wiki.train.raw")
    .filter(lambda x: tf.strings.length(x) > 100)
    .batch(PRETRAINING_BATCH_SIZE)
)

wiki_val_ds = (
    tf.data.TextLineDataset(wiki_dir + "wiki.valid.raw")
    .filter(lambda x: tf.strings.length(x) > 100)
    .batch(PRETRAINING_BATCH_SIZE)
)

# Take a peak at the sst-2 dataset.
print(sst_train_ds.unbatch().batch(4).take(1).get_single_element())

(<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'hide new secretions from the parental units ',
       b'contains no wit , only labored gags ',
       b'that loves its characters and communicates something rather beautiful about human nature ',
       b'remains utterly satisfied to remain the same throughout '],
      dtype=object)>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 1, 0], dtype=int32)>)


In [ ]:
# Create an appropriate baseline model to compare our model against

Epoch 1/5
   2105/Unknown 16s 3ms/step - accuracy: 0.6435 - loss: 0.6466

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2105/2105 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.6435 - loss: 0.6466 - val_accuracy: 0.7533 - val_loss: 0.5392
Epoch 2/5
2105/2105 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7506 - loss: 0.5398 - val_accuracy: 0.7790 - val_loss: 0.4893
Epoch 3/5
2105/2105 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7824 - loss: 0.4876 - val_accuracy: 0.7969 - val_loss: 0.4673
Epoch 4/5
2105/2105 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7987 - loss: 0.4547 - val_accuracy: 0.8002 - val_loss: 0.4571
Epoch 5/5
2105/2105 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8093 - loss: 0.4317 - val_accuracy: 0.8002 - val_loss: 0.4532


 # Masked Language Modelling

 We will be doing Masked Language Modelling for training as our dataset has no labels. Another way to train is by using Next Sentence Prediction (NSP) if we had labels. We will be focusing on MLM.

 **Definition**: *Masked language modeling predicts a masked token in a sequence, and the model can attend to tokens bidirectionally. This means the model has full access to the tokens on the left and right. Masked language modeling is great for tasks that require a good contextual understanding of an entire sequence. BERT is an example of a masked language model.*

In [ ]:
# What is word piece tokenizer doing?
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab_file,
    sequence_length=SEQ_LENGTH,
    lowercase=True,
    strip_accents=True,
)

# Setting mask_selection_length will trim or pad the mask outputs to shape
# (batch_size, PREDICTIONS_PER_SEQ).
masker = keras_nlp.layers.MaskedLMMaskGenerator(
    vocabulary_size=tokenizer.vocabulary_size(),
    mask_selection_rate=MASK_RATE,
    mask_selection_length=PREDICTIONS_PER_SEQ,
    mask_token_id=tokenizer.token_to_id("[MASK]"),
)

# Write a function that preprocess the data and returns features, labels, weights
def preprocess(inputs) -> Tuple[Dict[str, tf.Tensor], tf.Tensor, tf.Tensor]:
    features = {
        "token_ids": ...,
        "mask_positions": ...
    }
    return None

# We use prefetch() to pre-compute preprocessed batches on the fly on the CPU.
pretrain_ds = wiki_train_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

pretrain_val_ds = wiki_val_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

# Preview a single input example.
print(pretrain_val_ds.take(1).get_single_element())

InvalidArgumentError: ignored

# Encoder Only Transformer

We will be creating an encoder only block so that we only deal with self-attention. Create an encoder block using the following parameters:

In [ ]:
# Model params.
NUM_LAYERS = 3
MODEL_DIM = 256
INTERMEDIATE_DIM = 512
NUM_HEADS = 4
DROPOUT = 0.1
NORM_EPSILON = 1e-5

# What do each of these parameters represent?

In [ ]:
encoder_model = keras.Model(inputs, outputs)
encoder_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ token_and_position_embedding    │ (None, 128, 256)          │  7,846,400 │
│ (TokenAndPositionEmbedding)     │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ layer_normalization             │ (None, 128, 256)          │        512 │
│ (LayerNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout (Dropout)               │ (None, 128, 256)          │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_encoder             │ (None, 128, 256)          │    527,104 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_encoder_1           │ (None, 128, 256)          │    527,104 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_encoder_2           │ (None, 128, 256)          │    527,104 │
│ (TransformerEncoder)            │                           │            │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 9,428,224 (35.97 MB)

 Trainable params: 9,428,224 (35.97 MB)

 Non-trainable params: 0 (0.00 B)

Since this is a Masked Language Modelling Task, we will need to attach a Masked Model Head:

![masked_model](https://editor.analyticsvidhya.com/uploads/22971fig-3.png)



In [ ]:
# Training params.
PRETRAINING_LEARNING_RATE = 5e-4
PRETRAINING_EPOCHS = 8
FINETUNING_LEARNING_RATE = 5e-5
FINETUNING_EPOCHS = 3

In [ ]:
# Create the pretraining model by attaching a masked language model head.
inputs = {
    "token_ids": keras.Input(shape=(SEQ_LENGTH,), dtype=tf.int32, name="token_ids"),
    "mask_positions": keras.Input(
        shape=(PREDICTIONS_PER_SEQ,), dtype=tf.int32, name="mask_positions"
    ),
}


# Define and compile our pretraining model.
pretraining_model = keras.Model(inputs, outputs)

#Compile


In [ ]:
# Pretrain the model on our wiki text dataset.
pretraining_model.fit(
    pretrain_ds,
    validation_data=pretrain_val_ds,
    epochs=PRETRAINING_EPOCHS,
)

pretraining_model

Epoch 1/8
5857/5857 ━━━━━━━━━━━━━━━━━━━━ 994s 165ms/step - loss: 5.4042 - sparse_categorical_accuracy: 0.1437 - val_loss: 3.4693 - val_sparse_categorical_accuracy: 0.3540
Epoch 2/8
2220/5857 ━━━━━━━━━━━━━━━━━━━━ 9:44 161ms/step - loss: 3.6629 - sparse_categorical_accuracy: 0.3321

**Text Classification (e.g., Sentiment Analysis):**

+ **Fine-tuning**: The final hidden state corresponding to the [CLS] token (often the first token in the sequence) is taken as the aggregate representation of the input sequence. This representation is passed through a dense layer to produce a prediction for the classification task.
+ **Inference**: For a new input text, it's tokenized and passed through BERT. The output corresponding to the [CLS] token is processed by the dense layer, yielding a class label.

**Question Answering (e.g., SQuAD)**:

+ **Fine-tuning**: For each token in the input, the model predicts two scores - one for being the start of the answer span and one for being the end of the answer span.
+ **Inference**: For a new question and context pair, the model identifies the span in the context with the highest combined start and end scores as the predicted answer.

**Named Entity Recognition (NER)**:

+ **Fine-tuning**: Each token in the sequence is associated with a label (e.g., person, organization, etc. or O for 'no entity'). The hidden state of each token is processed by a dense layer to predict the label.
+ **Inference**: For a new input text, each token's output is processed to predict its entity label.

**Text Similarity or Pairwise Tasks**:

+ **Fine-tuning**: Similar to text classification, the [CLS] token representation can be used to predict a similarity score or class label for a pair of sentences.
+ **Inference**: For a new pair of texts, the combined representation (often based on the [CLS] token) is used to predict their relationship.
In all cases, the process is generally as follows:

# Word Masking

Training the language model in BERT is done by predicting 15% of the tokens in the input, that were randomly picked. These tokens are pre-processed as follows — 80% are replaced with a “[MASK]” token, 10% with a random word, and 10% use the original word. The intuition that led the authors to pick this approach is as follows (Thanks to Jacob Devlin from Google for the insight):

If we used [MASK] 100% of the time the model wouldn’t necessarily produce good token representations for non-masked words. The non-masked tokens were still used for context, but the model was optimized for predicting masked words.

If we used [MASK] 90% of the time and random words 10% of the time, this would teach the model that the observed word is never correct.

If we used [MASK] 90% of the time and kept the same word 10% of the time, then the model could just trivially copy the non-contextual embedding.

No ablation was done on the ratios of this approach, and it may have worked better with different ratios. In addition, the model performance wasn’t tested with simply masking 100% of the selected tokens